In [ ]:
import pickle as pkl 
import json
movie_ls = pkl.load(open("movie_ids.pkl",'rb'))


In [7]:
movie_ls = pkl.load(open("movie_ids.pkl",'rb'))

In [9]:
len(movie_ls)

1472

In [1]:
import pandas as pd
import json
import jsonlines
import re
test = pd.read_csv("./test.tsv",sep='\t')
train = pd.read_csv("./train.tsv",sep='\t')
valid = pd.read_csv("./dev.tsv",sep='\t')


In [2]:
test_db = []
with open("./test_data_dbpedia.jsonl",'r') as f:
    test_db_lines = f.readlines()
    for line in test_db_lines:
        test_db.append(json.loads(line))
train_db = []
with open("./train_data_dbpedia.jsonl",'r') as f:
    train_db_lines = f.readlines()
    for line in train_db_lines:
        train_db.append(json.loads(line))
valid_db = []
with open("./valid_data_dbpedia.jsonl",'r') as f:
    valid_db_lines = f.readlines()
    for line in valid_db_lines:
        valid_db.append(json.loads(line))

In [3]:
moviename2entity = {}
num =0
text_dict_new = {}
movienamenontime2entity = {}

movie_set = set()

for line in test_db:
    for dialog in line:
        text = dialog['text']
        dic = {}
        dic['entity_link'] = dialog['entity_link']
        dic['entity_name'] = dialog['entity_name']
        dic['movie_link'] = dialog['movie_link']
        dic['movie_name'] = dialog['movie_name']
        dic['movie'] = dialog['movie']
        for name,entity in zip(dialog['movie'],dialog['movie_link']):
            moviename2entity[name.lower()] = entity
            movie_set.add(entity)
        for name,entity in zip(dialog['movie_name'],dialog['movie_link']):
            movienamenontime2entity[name.lower().replace(" ","")] = entity
        if text in text_dict_new.keys():
            num +=1
        text_dict_new[text] = dic
for line in valid_db:
    for dialog in line:
        text = dialog['text']
        dic = {}
        dic['entity_link'] = dialog['entity_link']
        dic['entity_name'] = dialog['entity_name']
        dic['movie_link'] = dialog['movie_link']
        dic['movie_name'] = dialog['movie_name']
        for name,entity in zip(dialog['movie'],dialog['movie_link']):
            movie_set.add(entity)
            moviename2entity[name.lower()] = entity
        for name,entity in zip(dialog['movie_name'],dialog['movie_link']):
            movienamenontime2entity[name.lower().replace(" ","")] = entity
        if text in text_dict_new.keys():
            num +=1
        text_dict_new[text] = dic
for line in train_db:
    for dialog in line:
        text = dialog['text']
        dic = {}
        dic['entity_link'] = dialog['entity_link']
        dic['entity_name'] = dialog['entity_name']
        dic['movie_link'] = dialog['movie_link']
        dic['movie_name'] = dialog['movie_name']
        for name,entity in zip(dialog['movie'],dialog['movie_link']):
            movie_set.add(entity)
            moviename2entity[name.lower()] = entity
        for name,entity in zip(dialog['movie_name'],dialog['movie_link']):
            movienamenontime2entity[name.lower().replace(" ","")] = entity
        if text in text_dict_new.keys():
            num +=1
        text_dict_new[text] = dic
#     # break

In [9]:
moviename_entity_df = pd.DataFrame(columns=['name','entity_link'])
for key in moviename2entity.keys():
    moviename_entity_df.loc[len(moviename_entity_df)] = {
        "name":key,
        "entity_link":moviename2entity[key]
    }

In [11]:
moviename_entity_df.to_csv("inspired_movie_entitylink.csv",index=False)

In [44]:
# num_not_in_data = 0
# movie_not_in_data = []

# last_dialog_id = None
# last_id = None
# last_text = ""
# movieMentions = {}
# messages = []
# movie_label = {}
# test_data = []

# for dialog_id,df in test.groupby("dialog_id"):
#     last_text = ""
#     text = ""
#     for index,row in df.iterrows():
#         if row['speaker'] == last_id:
#             text += last_text + " "
#         else:
#             dic = {}
#             dic['text'] = text
#             dic['role'] = row['speaker']
#             messages.append(dic)
#             text = ""
#             # messages['movieMentions'] = movieMentions
#             pass
#         if "QUOTATION_MARK" in row['text']:
#             movie = re.findall("QUOTATION_MARK.*QUOTATION_MARK",row['text'])[0].replace("QUOTATION_MARK","").lower()
#             if movie in moviename2entity.keys():
#                 movieMentions[movie] = moviename2entity[movie]
#                 movie_label[movie] = row['fine_label']
#             else:
#                 num_not_in_data +=1
#                 movie_not_in_data.append(movie)
#         last_text = row['text'].replace("QUOTATION_MARK","")
#         last_dialog_id = row['dialog_id']
#         last_id = row['speaker']

#     text += last_text

#     temp_dic = {}
#     temp_dic['dialog_id'] = dialog_id
#     temp_dic['movieMentions'] = movieMentions
#     temp_dic['messages'] = messages
#     temp_dic['movie_label'] = movie_label
#     test_data.append(temp_dic)

#     break

In [45]:
last_dialog_id = test.iloc[0]['dialog_id']
movieMentions = {}
movie_label = {}

id_not_in_data = set()
num_not_in_data = 0
movie_not_in_data = []

test_data = []
id = 0
for index,row in test.iterrows():
    # movie = row['movies']
    
    text = row['text_with_placeholder']
    movie_dict = eval(row['movie_dict'])
    id2moive = {}
    for k,v in movie_dict.items():
        id2moive[str(v)] = k
    
    if row['dialog_id']!=last_dialog_id:
        temp_dic = {}
        temp_dic['movieMentions']= movieMentions
        temp_dic['movie_label']= movie_label
        temp_dic['messages'] = test_db[id]
        test_data.append(temp_dic)

        movieMentions = {}
        movie_label = {}
        id +=1
    # if str(movie)=='nan':
    #     last_dialog_id = row['dialog_id']
    #     continue
    # movie = movie.lower()
    # if movie in moviename2entity.keys():
    #     movieMentions[movie] = moviename2entity[movie]
    #     movie_label[movie] = row['fine_label']
    # else:
    #     num_not_in_data +=1
    #     movie_not_in_data.append(movie)
    #     id_not_in_data.add(id)
    if "MOVIE_TITLE" in text:
        movie_id = re.findall("MOVIE_TITLE_[0-9]*",text)[0].replace("MOVIE_TITLE_","")
        movie = id2moive[movie_id].lower()
        if movie in moviename2entity.keys():
            movieMentions[movie] = moviename2entity[movie]
            movie_label[moviename2entity[movie]] = row['fine_label']
        else:
            num_not_in_data +=1
            movie_not_in_data.append(movie)
            id_not_in_data.add(id)
        # break
    last_dialog_id = row['dialog_id']
temp_dic = {}
temp_dic['messages'] = test_db[id]
temp_dic['movieMentions']= movieMentions
temp_dic['movie_label']= movie_label
test_data.append(temp_dic)

In [46]:
last_dialog_id = train.iloc[0]['dialog_id']
movieMentions = {}
movie_label = {}

id_not_in_data = set()
num_not_in_data = 0
movie_not_in_data = []

train_data = []
id = 0
for index,row in train.iterrows():
    text = row['text_with_placeholder']
    try:
        movie_dict = eval(row['movie_dict'])
    except:
        movie_dict = {}
    id2moive = {}
    for k,v in movie_dict.items():
        id2moive[str(v)] = k
    
    if row['dialog_id']!=last_dialog_id:
        temp_dic = {}
        temp_dic['movieMentions']= movieMentions
        temp_dic['movie_label']= movie_label
        temp_dic['messages'] = train_db[id]
        train_data.append(temp_dic)

        movieMentions = {}
        movie_label = {}
        id +=1

    if "MOVIE_TITLE" in text:
        movie_id = re.findall("MOVIE_TITLE_[0-9]*",text)[0].replace("MOVIE_TITLE_","")
        movie = id2moive[movie_id].lower()
        if movie in moviename2entity.keys():
            movieMentions[movie] = moviename2entity[movie]
            movie_label[moviename2entity[movie]] = row['fine_label']
        else:
            num_not_in_data +=1
            movie_not_in_data.append(movie)
            id_not_in_data.add(id)
        # break
    last_dialog_id = row['dialog_id']
temp_dic = {}
temp_dic['messages'] = train_db[id]
temp_dic['movieMentions']= movieMentions
temp_dic['movie_label']= movie_label
train_data.append(temp_dic)

In [47]:
last_dialog_id = valid.iloc[0]['dialog_id']
movieMentions = {}
movie_label = {}

id_not_in_data = set()
num_not_in_data = 0
movie_not_in_data = []

valid_data = []
id = 0
for index,row in valid.iterrows():
    text = row['text_with_placeholder']
    try:
        movie_dict = eval(row['movie_dict'])
    except:
        movie_dict = {}
    id2moive = {}
    for k,v in movie_dict.items():
        id2moive[str(v)] = k
    
    if row['dialog_id']!=last_dialog_id:
        temp_dic = {}
        temp_dic['movieMentions']= movieMentions
        temp_dic['movie_label']= movie_label
        temp_dic['messages'] = valid_db[id]
        valid_data.append(temp_dic)

        movieMentions = {}
        movie_label = {}
        id +=1

    if "MOVIE_TITLE" in text:
        movie_id = re.findall("MOVIE_TITLE_[0-9]*",text)[0].replace("MOVIE_TITLE_","")
        movie = id2moive[movie_id].lower()
        if movie in moviename2entity.keys():
            movieMentions[movie] = moviename2entity[movie]
            movie_label[moviename2entity[movie]] = row['fine_label']
        else:
            num_not_in_data +=1
            movie_not_in_data.append(movie)
            id_not_in_data.add(id)
        # break
    last_dialog_id = row['dialog_id']
temp_dic = {}
temp_dic['messages'] = valid_db[id]
temp_dic['movieMentions']= movieMentions
temp_dic['movie_label']= movie_label
valid_data.append(temp_dic)

In [2]:
import pickle
import json

In [3]:
with open('./dbpedia_subkg.json','r') as f:
    kg = json.loads(f.read())
with open('./entity2id.json','r') as f:
    entity2id = json.loads(f.read())


In [38]:
movie_ids = []
for movie_entity in movie_set:
    movie_ids.append(entity2id[movie_entity])

In [16]:
for key in entity2id.keys():
    if entity2id[key]==137:
        temp_key = key
        break

In [17]:
for key in moviename2entity.keys():
    if moviename2entity[key]==temp_key:
        break


In [4]:
sub_kg = {}
for item in kg.items():
    sub_kg[int(item[0])] = item[1]

In [5]:
sub_kg

{14127: [[12, 14127],
  [10, 6691],
  [6, 9970],
  [1, 16768],
  [11, 1491],
  [4, 235],
  [3, 9970],
  [3, 2154],
  [0, 7685],
  [0, 2386],
  [0, 1389],
  [0, 10176],
  [0, 9331],
  [5, 6691]],
 2166: [[12, 2166],
  [2, 7957],
  [10, 9690],
  [6, 9509],
  [1, 14688],
  [11, 3225],
  [11, 4401],
  [4, 1590],
  [4, 3893],
  [4, 12658],
  [4, 4552],
  [0, 15568],
  [0, 4341],
  [0, 15012],
  [0, 2930],
  [5, 688],
  [5, 13299]],
 5404: [[12, 5404],
  [2, 10704],
  [10, 4159],
  [6, 1297],
  [1, 10017],
  [11, 11298],
  [4, 9520],
  [4, 200],
  [4, 11465],
  [3, 10938],
  [3, 7636],
  [3, 16209],
  [3, 17228],
  [0, 11206],
  [0, 1721],
  [0, 15914],
  [0, 14353],
  [0, 1719]],
 6520: [[12, 6520],
  [10, 3259],
  [6, 12903],
  [3, 14500],
  [0, 11864],
  [0, 92],
  [0, 17113],
  [0, 8954],
  [0, 16778],
  [0, 6974]],
 3291: [[12, 3291],
  [10, 14424],
  [10, 10335],
  [6, 3076],
  [1, 300],
  [11, 5450],
  [4, 4919],
  [3, 1148],
  [3, 4133],
  [0, 16945],
  [0, 1558],
  [0, 7211],
  [0, 

In [81]:
sub_kg = {}
for item in kg.items():
    sub_kg[int(item[0])] = item[1]
id2entity = {}
for k,v in entity2id.items():
    id2entity[v] = k

In [82]:
# import jsonline
# with jsonline.open("./train_data.jsonl",'wb') as f:
#     for line in train_data:
#         f.write(line)
# with jsonline.open("./valid_data.jsonl",'wb') as f:
#     for line in valid_data:
#         f.write(line)
# with jsonline.open("./test_data.jsonl",'wb') as f:
#     for line in test_data:
#         f.write(line)

In [83]:
with open("./train_data.pkl",'wb') as f:
    pickle.dump(train_data, f)
with open("./valid_data.pkl",'wb') as f:
    pickle.dump(valid_data, f)
with open("./test_data.pkl",'wb') as f:
    pickle.dump(test_data, f)
with open("./subkg.pkl",'wb') as f:
    pickle.dump(sub_kg, f)
with open("./entity2id.pkl",'wb') as f:
    pickle.dump(entity2id, f)
with open("./id2entity.pkl",'wb') as f:
    pickle.dump(id2entity, f)
with open("./movie_ids.pkl",'wb') as f:
    pickle.dump(movie_ids, f)
with open("./moviename2entity.pkl",'wb') as f:
    pickle.dump(moviename2entity, f)

In [34]:
len(entity2id)

17320

In [26]:
# num_not_in_data = 0
# movie_not_in_data = []

# last_dialog_id = None
# last_id = None
# last_text = ""
# movieMentions = {}
# messages = []
# movie_label = {}
# test_data = []
# for index,row in test.iterrows():
#     text = row['text']
#     if row['dialog_id']!=last_dialog_id:
#         temp_dic = {}
#         temp_dic['dialog_id'] = row['dialog_id']
#         temp_dic['movieMentions'] = movieMentions
#         temp_dic['messages'] = messages
#         temp_dic['movie_label'] = movie_label
#         test_data.append(temp_dic)

#         movieMentions = {}
#         messages = []
#         movie_label = {}
#     if row['speaker'] == last_id or row['turn_id']==1:
#         last_text += text + " "
#     else:
#         dic = {}
#         dic['text'] = last_text[:-1]
#         dic['role'] = row['speaker']
#         messages.append(dic)
#         # messages['movieMentions'] = movieMentions
#         pass
#     last_id = row['speaker']
#     if "QUOTATION_MARK" in text:
#         movie = re.findall("QUOTATION_MARK.*QUOTATION_MARK",text)[0].replace("QUOTATION_MARK","").lower()
#         if movie in moviename2entity.keys():
#             movieMentions[movie] = moviename2entity[movie]
#             movie_label[movie] = row['fine_label']
#         else:
#             num_not_in_data +=1
#             movie_not_in_data.append(movie)
#     last_text = row['text'].replace("QUOTATION_MARK","")
#     last_dialog_id = row['dialog_id']

In [120]:
# import requests

# DBPEDIA_SPOTLIGHT_ADDR = "http://localhost:2222/rest/annotate"


# def _text2entities(text, confidence):
#     headers = {"Accept": "application/json"}
#     data = {"text": text, "confidence": confidence}

#     response = requests.get(url=DBPEDIA_SPOTLIGHT_ADDR, params=data, headers=headers)
#     response = response.json()
#     return (
#         [f"<{x['@URI']}>" for x in response["Resources"]]
#         if "Resources" in response
#         else []
#     )


# if __name__ == '__main__':
#     print(_text2entities("The Child (2005)", 0.3))



['<http://dbpedia.org/resource/Grogu>']
